In [1]:
import os
import pandas as pd

# ---------------------------------------
# 0) Config
# ---------------------------------------
INPUT_ROOT = "/kaggle/input"
OUTPUT_ROOT = "/kaggle/working/excel_csv"
EXCEL_EXTS = {".xls", ".xlsx", ".xlsm"}
CSV_EXTS = {".csv"}

os.makedirs(OUTPUT_ROOT, exist_ok=True)

# ---------------------------------------
# 1) Helper: Print folder tree
# ---------------------------------------
def print_tree(root_path):
    """
    Print a simple 'tree' of the given root directory.
    """
    print(f"\n[DIR TREE] {root_path}")
    if not os.path.exists(root_path):
        print("  (path does not exist)")
        return

    def _walk(path, prefix=""):
        name = os.path.basename(path.rstrip(os.sep))
        if not name:  # root like "/"
            name = path
        print(prefix + name + "/")
        try:
            entries = sorted(os.listdir(path))
        except PermissionError:
            print(prefix + "    [Permission denied]")
            return

        for entry in entries:
            full = os.path.join(path, entry)
            if os.path.isdir(full):
                _walk(full, prefix + "    ")
            else:
                print(prefix + "    " + entry)

    _walk(root_path)

# ---------------------------------------
# 2) Show the INPUT folder tree
# ---------------------------------------
print_tree(INPUT_ROOT)

# ---------------------------------------
# 3) Walk /kaggle/input and process Excel + CSV
#    - Excel: convert → CSV in OUTPUT_ROOT
#    - CSV:  copy (read & write) → OUTPUT_ROOT
# ---------------------------------------
summaries = []

for dirpath, _, filenames in os.walk(INPUT_ROOT):
    for fname in filenames:
        ext = os.path.splitext(fname)[1].lower()
        if ext not in EXCEL_EXTS and ext not in CSV_EXTS:
            continue  # skip non-tabular files

        src_path = os.path.join(dirpath, fname)

        # Keep the same relative folder structure under OUTPUT_ROOT
        rel_dir = os.path.relpath(dirpath, INPUT_ROOT)
        out_dir = os.path.join(OUTPUT_ROOT, rel_dir)
        os.makedirs(out_dir, exist_ok=True)

        base = os.path.splitext(fname)[0]
        out_csv_path = os.path.join(out_dir, base + ".csv")

        # Decide how to read
        if ext in EXCEL_EXTS:
            print(f"\n[INFO] Converting Excel → CSV")
            print(f"       From: {src_path}")
            print(f"       To:   {out_csv_path}")
            df = pd.read_excel(src_path)
        else:  # CSV_EXTS
            print(f"\n[INFO] Copying CSV → OUTPUT")
            print(f"       From: {src_path}")
            print(f"       To:   {out_csv_path}")
            df = pd.read_csv(src_path)

        # Write CSV to output directory
        df.to_csv(out_csv_path, index=False)

        # Store summary
        summary = {
            "csv_path": out_csv_path,
            "n_rows": df.shape[0],
            "n_cols": df.shape[1],
            "columns": list(df.columns),
            "dtypes": df.dtypes.astype(str).to_dict(),
            "missing_total": int(df.isna().sum().sum()),
            "preview": df.head(3).to_string(index=False)
        }
        summaries.append(summary)

# ---------------------------------------
# 4) Print summary for each CSV
# ---------------------------------------
print("\n" + "="*70)
print("BRIEF SUMMARY OF CREATED CSV FILES")
print("="*70)

if not summaries:
    print("No Excel or CSV files were processed under /kaggle/input.")
else:
    for s in summaries:
        print(f"\nFile: {s['csv_path']}")
        print(f"  Shape: {s['n_rows']} rows × {s['n_cols']} columns")
        print(f"  Columns: {s['columns']}")
        print(f"  Dtypes: {s['dtypes']}")
        print(f"  Total missing values: {s['missing_total']}")
        print("  First 3 rows:")
        print(s["preview"])

# ---------------------------------------
# 5) Show the OUTPUT folder tree
# ---------------------------------------
print_tree(OUTPUT_ROOT)

print("\n[DONE] All Excel and CSV files copied/converted to OUTPUT with summaries.")



[DIR TREE] /kaggle/input
input/
    amazon-pet-supplies-data/
        amazon_pet_supplies_dataset_sample.csv
    chewy-data/
        chewy_scraper_sample.csv
    e-commerce-pet-supplies-dataset/
        aliexpress_pet_supplies.csv
    pet-food-customer-orders-online/
        pet_food_customer_orders.csv
    pet-store-records-2020/
        pet_store_records_2020.csv
    sales-data-shop-pet/
        metrics_2023-01-01_to_2023-08-10_targeted_interest.csv
    us-pet-food-supply-stores/
        US Pet Food  Supply Stores.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/pet-store-records-2020/pet_store_records_2020.csv
       To:   /kaggle/working/excel_csv/pet-store-records-2020/pet_store_records_2020.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/pet-food-customer-orders-online/pet_food_customer_orders.csv
       To:   /kaggle/working/excel_csv/pet-food-customer-orders-online/pet_food_customer_orders.csv


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan


[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/e-commerce-pet-supplies-dataset/aliexpress_pet_supplies.csv
       To:   /kaggle/working/excel_csv/e-commerce-pet-supplies-dataset/aliexpress_pet_supplies.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/amazon-pet-supplies-data/amazon_pet_supplies_dataset_sample.csv
       To:   /kaggle/working/excel_csv/amazon-pet-supplies-data/amazon_pet_supplies_dataset_sample.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/us-pet-food-supply-stores/US Pet Food  Supply Stores.csv
       To:   /kaggle/working/excel_csv/us-pet-food-supply-stores/US Pet Food  Supply Stores.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/sales-data-shop-pet/metrics_2023-01-01_to_2023-08-10_targeted_interest.csv
       To:   /kaggle/working/excel_csv/sales-data-shop-pet/metrics_2023-01-01_to_2023-08-10_targeted_interest.csv

[INFO] Copying CSV → OUTPUT
       From: /kaggle/input/chewy-data/chewy_scraper_sample.csv
       To:   /ka

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

In [2]:
    pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 61.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ============================================================
# Pet-Shop Recommender: Preprocessing, EDA, Multi-Model Compare
# ============================================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

# If needed on Kaggle:
# !pip install implicit

import implicit

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ------------------------------------------------------------
# 0) LOAD ALL DATA
# ------------------------------------------------------------
BASE = "/kaggle/working/excel_csv"

paths = {
    "tails":   os.path.join(BASE, "pet-food-customer-orders-online", "pet_food_customer_orders.csv"),
    "store":   os.path.join(BASE, "pet-store-records-2020", "pet_store_records_2020.csv"),
    "chewy":   os.path.join(BASE, "chewy-data", "chewy_scraper_sample.csv"),
    "amz":     os.path.join(BASE, "amazon-pet-supplies-data", "amazon_pet_supplies_dataset_sample.csv"),
    "ali":     os.path.join(BASE, "e-commerce-pet-supplies-dataset", "aliexpress_pet_supplies.csv"),
    "usst":    os.path.join(BASE, "us-pet-food-supply-stores", "US Pet Food  Supply Stores.csv"),
    "metrics": os.path.join(BASE, "sales-data-shop-pet", "metrics_2023-01-01_to_2023-08-10_targeted_interest.csv"),
}

print("Loading CSVs...")
tails   = pd.read_csv(paths["tails"])
store   = pd.read_csv(paths["store"])
chewy   = pd.read_csv(paths["chewy"])
amz     = pd.read_csv(paths["amz"])
ali     = pd.read_csv(paths["ali"])
usst    = pd.read_csv(paths["usst"])
metrics = pd.read_csv(paths["metrics"])

dfs = {
    "tails": tails,
    "store": store,
    "chewy": chewy,
    "amz":   amz,
    "ali":   ali,
    "usst":  usst,
    "metrics": metrics
}

for name, df in dfs.items():
    print(f"{name:8s} → {df.shape[0]:6d} rows × {df.shape[1]:3d} cols")

# ------------------------------------------------------------
# 1) QUICK EDA
# ------------------------------------------------------------
print("\n=== BASIC EDA ===")

# 1.1 Tails orders
print("\n[TAILS] pet_food_customer_orders overview")
print(tails[['customer_id', 'pet_id', 'pet_order_number', 'pet_food_tier',
             'wet_tray_size', 'wet_trays']].head())

print("\n[TAILS] pet_food_tier value counts:")
print(tails['pet_food_tier'].value_counts(dropna=False).head(10))

print("\n[TAILS] wet_tray_size value counts:")
print(tails['wet_tray_size'].value_counts(dropna=False).head(10))

# 1.2 Store records
print("\n[STORE] pet_store_records_2020 overview")
print(store[['product_id', 'product_category', 'vendor_id', 'country',
             'pet_type', 'pet_size', 'rating', 're_buy']].head())

print("\n[STORE] product_category value counts:")
print(store['product_category'].value_counts().head(10))

print("\n[STORE] pet_type value counts:")
print(store['pet_type'].value_counts().head(10))

# 1.3 Chewy / Amazon / AliExpress (catalogs)
print("\n[CHEWY] sample:")
print(chewy[['name', 'brand', 'breadcrumb', 'Price', 'average_rating']].head(3))

print("\n[AMAZON] sample:")
print(amz[['title', 'brand', 'price', 'currency', 'breadcrumbs']].head(3))

print("\n[ALIEXPRESS] sample:")
print(ali.head(3))

# 1.4 US pet food stores
print("\n[US Stores] sample:")
print(usst[['Business Name', 'City', 'State', 'Main Line of Business']].head(3))

# 1.5 Metrics (ads / marketing)
print("\n[Metrics] columns:")
print(metrics.columns[:15])
print("\n[Metrics] sample:")
print(metrics.head(3))
# ------------------------------------------------------------
# 2) BUILD INTERACTION TABLE (TAILS + STORE) — MORE GRANULAR
# ------------------------------------------------------------
# ------------------------------------------------------------
# 2) BUILD INTERACTION TABLE (TAILS + STORE) — MORE GRANULAR
# ------------------------------------------------------------
print("\n=== BUILD INTERACTIONS ===")

# ---- 2.1 Tails interactions (customer ↔ detailed food-type) ----
tails_small = tails.copy()

# Fill important columns
for col, default in [
    ('pet_food_tier', 'unknown'),
    ('wet_tray_size', 'none'),
    ('pet_life_stage_at_order', 'unknown'),
    ('pet_breed_size', 'unknown'),
]:
    if col in tails_small.columns:
        tails_small[col] = tails_small[col].fillna(default).astype(str)
    else:
        # if a column is missing in your CSV, just create it as unknown
        tails_small[col] = default

# item_key: now 4 attributes → many more unique "recipes"
tails_small['item_key'] = (
    "tails_"
    + tails_small['pet_food_tier'] + "|"
    + tails_small['wet_tray_size'] + "|"
    + tails_small['pet_life_stage_at_order'] + "|"
    + tails_small['pet_breed_size']
)

# user_key: customer
tails_small['user_key'] = "cust_" + tails_small['customer_id'].astype(str)

# weight from usage: we keep it simple but not just 1
#   base = max(wet_trays, 1)
#   small bonus if they've placed more orders for this pet's wet food
tails_small['wet_trays_filled'] = tails_small['wet_trays'].fillna(0)
base = tails_small['wet_trays_filled'].clip(lower=1)

if 'orders_since_first_wet_trays_order' in tails_small.columns:
    bonus = tails_small['orders_since_first_wet_trays_order'].fillna(0) * 0.1
else:
    bonus = 0

tails_small['weight'] = (base + bonus).astype(float)

tails_inter = (
    tails_small[['user_key', 'item_key', 'weight']]
    .groupby(['user_key', 'item_key'], as_index=False)['weight']
    .sum()
)

print("Interactions from Tails:", tails_inter.shape)

# ---- 2.2 Store interactions (vendor ↔ product) ----
store_small = store.copy()
store_small['item_key'] = "store_" + store_small['product_id'].astype(str)
store_small['user_key'] = "vendor_" + store_small['vendor_id'].astype(str)

# use re_buy as signal (repeated purchase), or at least 1 if anything
store_small['weight'] = 1.0 + store_small['re_buy'].fillna(0)
store_inter = (
    store_small[['user_key', 'item_key', 'weight']]
    .groupby(['user_key', 'item_key'], as_index=False)['weight']
    .sum()
)

print("Interactions from Store:", store_inter.shape)

# ---- 2.3 Combine ----
interactions = pd.concat(
    [
        tails_inter[['user_key', 'item_key', 'weight']],
        store_inter[['user_key', 'item_key', 'weight']],
    ],
    ignore_index=True
)
print("Total interactions (raw):", interactions.shape)

# ---- 2.4 Filter low-activity (SOFTER on items) ----
def filter_by_min_interactions(df, min_user=2, min_item=1):
    """
    Keep users with >= min_user interactions;
    keep items with >= min_item interactions.
    (min_item=1 means we don't aggressively kill rare products.)
    """
    df = df.copy()
    while True:
        uc = df['user_key'].value_counts()
        ic = df['item_key'].value_counts()
        before = df.shape[0]
        df = df[df['user_key'].isin(uc[uc >= min_user].index)]
        df = df[df['item_key'].isin(ic[ic >= min_item].index)]
        after = df.shape[0]
        if after == before:
            break
    return df

interactions = filter_by_min_interactions(interactions,
                                          min_user=2, min_item=1)
print("Total interactions (filtered):", interactions.shape)

# small EDA on interactions
print("\nNumber of distinct users:", interactions['user_key'].nunique())
print("Number of distinct items:", interactions['item_key'].nunique())
print("Top 10 most popular items:\n",
      (interactions.groupby('item_key')['weight'].sum()
                  .sort_values(ascending=False)
                  .head(10)))


# ------------------------------------------------------------
# 3) ENCODE USERS/ITEMS
# ------------------------------------------------------------
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

interactions['user_id'] = user_encoder.fit_transform(interactions['user_key'])
interactions['item_id'] = item_encoder.fit_transform(interactions['item_key'])

n_users = interactions['user_id'].nunique()
n_items = interactions['item_id'].nunique()

print(f"\nEncoded users: {n_users}, items: {n_items}")

# ------------------------------------------------------------
# 4) TRAIN–TEST SPLIT BY USER
# ------------------------------------------------------------
def train_test_split_by_user(df, test_size_per_user=1, random_state=42):
    df = df.sort_values('user_id')
    test_rows = []
    train_rows = []
    rng = np.random.RandomState(random_state)

    for uid, group in df.groupby('user_id'):
        if len(group) <= test_size_per_user:
            train_rows.append(group)
            continue
        test_idx = rng.choice(group.index.values,
                              size=test_size_per_user,
                              replace=False)
        test_mask = group.index.isin(test_idx)
        test_rows.append(group[test_mask])
        train_rows.append(group[~test_mask])

    train_df = pd.concat(train_rows, ignore_index=True)
    test_df  = pd.concat(test_rows,  ignore_index=True)
    return train_df, test_df

train_df, test_df = train_test_split_by_user(
    interactions, test_size_per_user=1, random_state=RANDOM_STATE
)

print("Train interactions:", train_df.shape)
print("Test interactions:", test_df.shape)

# ------------------------------------------------------------
# 5) BUILD SPARSE MATRICES
# ------------------------------------------------------------
def build_sparse_matrix(df, n_users, n_items):
    return sp.coo_matrix(
        (df['weight'].astype(float),
         (df['user_id'].astype(int), df['item_id'].astype(int))),
        shape=(n_users, n_items)
    ).tocsr()

train_mat = build_sparse_matrix(train_df, n_users, n_items)
test_mat  = build_sparse_matrix(test_df,  n_users, n_items)

print("Sparse shapes — train:", train_mat.shape, "test:", test_mat.shape)

# ------------------------------------------------------------
# 6) BUILD UNIFIED ITEM CATALOG (USES CHEWY/AMZ/ALI TOO)
# ------------------------------------------------------------
# --- core items from tails & store ---
cat_rows = []

# tails
temp_tails_items = tails_small[['item_key', 'pet_food_tier', 'wet_tray_size']].drop_duplicates()
temp_tails_items['source'] = 'tails'
temp_tails_items = temp_tails_items.rename(columns={
    'pet_food_tier': 'category',
    'wet_tray_size': 'size'
})
cat_rows.append(temp_tails_items[['item_key', 'source', 'category', 'size']])

# store
temp_store_items = store_small[['item_key', 'product_category', 'pet_type', 'pet_size']].drop_duplicates()
temp_store_items['source'] = 'store'
temp_store_items = temp_store_items.rename(columns={
    'product_category': 'category',
    'pet_size': 'size'
})
cat_rows.append(temp_store_items[['item_key', 'source', 'category', 'pet_type', 'size']])

catalog_core = pd.concat(cat_rows, ignore_index=True).drop_duplicates('item_key')

# --- extra items from Chewy / Amazon / AliExpress ---
chewy_items = pd.DataFrame({
    'item_key': "chewy_" + chewy['uniq_id'].astype(str),
    'source':   'chewy',
    'name':     chewy['name'],
    'brand':    chewy['brand'],
    'category': chewy['breadcrumb']
})

amz_items = pd.DataFrame({
    'item_key': "amz_" + amz['asin'].astype(str),
    'source':   'amazon',
    'name':     amz['title'],
    'brand':    amz['brand'],
    'category': amz['breadcrumbs']
})

ali_items = pd.DataFrame({
    'item_key': "ali_" + ali.index.astype(str),
    'source':   'aliexpress',
    'name':     ali['title'],
    'brand':    np.nan,
    'category': 'AliExpressPetSupplies'
})

catalog_extra = pd.concat([chewy_items, amz_items, ali_items], ignore_index=True)

items_catalog = pd.concat([catalog_core, catalog_extra], ignore_index=True)
items_catalog = items_catalog.drop_duplicates('item_key')

items_catalog['item_id'] = -1
mask_known = items_catalog['item_key'].isin(item_encoder.classes_)
items_catalog.loc[mask_known, 'item_id'] = item_encoder.transform(
    items_catalog.loc[mask_known, 'item_key']
)

print("\nUnified item catalog size:", items_catalog.shape)
print(items_catalog.head())
# ------------------------------------------------------------
# 6b) SIMULATE INTERACTIONS FOR CHEWY / AMAZON / ALI
# ------------------------------------------------------------

core_sources = ["tails", "store"]
ext_sources  = ["chewy", "amazon", "aliexpress"]  # as defined above

core_mask = items_catalog["source"].isin(core_sources)
ext_mask  = items_catalog["source"].isin(ext_sources)

print("Core items:", core_mask.sum(), "External items:", ext_mask.sum())

# 6b.1 Build simple text for each item (for content similarity)
def build_item_text(row):
    parts = []
    for col in ["category", "size", "pet_type", "name", "brand"]:
        if col in row and pd.notna(row[col]):
            parts.append(str(row[col]))
    return " ".join(parts)

items_catalog["text"] = items_catalog.apply(build_item_text, axis=1).fillna("")

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(items_catalog["text"])

core_idx = np.where(core_mask)[0]
ext_idx  = np.where(ext_mask)[0]

X_core = X[core_idx]
X_ext  = X[ext_idx]

from sklearn.metrics.pairwise import cosine_similarity
sim_ext_core = cosine_similarity(X_ext, X_core)  # [n_ext, n_core]
print("Similarity matrix shape:", sim_ext_core.shape)

# 6b.2 Map core items -> real users
item_to_users = (
    interactions
    .groupby("item_key")["user_key"]
    .apply(list)
    .to_dict()
)
print("Number of core items with real users:", len(item_to_users))

# 6b.3 Generate synthetic interactions
rng = np.random.RandomState(123)

K_SIM = 5              # top-K similar core items
MAX_USERS_PER_EXT = 20 # max synthetic users per external item
MIN_WEIGHT = 0.1       # avoid 0
scale_weight = 3.0     # similarity -> implicit weight

synthetic_rows = []  # (user_key, item_key, weight)

for ext_pos, row_idx in enumerate(ext_idx):
    ext_row = items_catalog.iloc[row_idx]
    ext_key = ext_row["item_key"]
    sims = sim_ext_core[ext_pos]
    if np.all(sims <= 0):
        continue

    top_core_local = np.argsort(-sims)[:K_SIM]
    used_users = set()

    for local_idx in top_core_local:
        core_global_idx = core_idx[local_idx]
        core_key = items_catalog.iloc[core_global_idx]["item_key"]
        users = item_to_users.get(core_key, [])
        if not users:
            continue

        n_users = len(users)
        sample_size = max(1, min(n_users, n_users // 10))
        sampled_users = rng.choice(users, size=sample_size, replace=False)

        sim_val = float(sims[local_idx])  # [0,1]
        base_w  = max(MIN_WEIGHT, sim_val * scale_weight)

        for u in sampled_users:
            if u in used_users:
                continue
            used_users.add(u)
            synthetic_rows.append((u, ext_key, base_w))

        if len(used_users) >= MAX_USERS_PER_EXT:
            break

print("Synthetic interactions generated:", len(synthetic_rows))

synthetic_df = pd.DataFrame(
    synthetic_rows,
    columns=["user_key", "item_key", "weight"]
)

# 6b.4 Merge synthetic with real interactions
interactions_aug = pd.concat(
    [interactions[["user_key", "item_key", "weight"]], synthetic_df],
    ignore_index=True
)

print("Original interactions:", interactions.shape)
print("Augmented interactions:", interactions_aug.shape)

# reuse your existing filter_by_min_interactions (already defined above)
interactions_aug = filter_by_min_interactions(
    interactions_aug,
    min_user=2,
    min_item=1   # keep rare external products
)
print("Augmented interactions after filtering:", interactions_aug.shape)

# From now on, we work with the augmented interactions
interactions = interactions_aug

# ------------------------------------------------------------
# 7) ENCODE USERS/ITEMS (AFTER AUGMENTATION)
# ------------------------------------------------------------
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

interactions["user_id"] = user_encoder.fit_transform(interactions["user_key"])
interactions["item_id"] = item_encoder.fit_transform(interactions["item_key"])

n_users = interactions["user_id"].nunique()
n_items = interactions["item_id"].nunique()

print(f"\nEncoded users (augmented): {n_users}, items: {n_items}")

# ------------------------------------------------------------
# 8) TRAIN–TEST SPLIT BY USER (SAME AS BEFORE)
# ------------------------------------------------------------
train_df, test_df = train_test_split_by_user(
    interactions, test_size_per_user=1, random_state=RANDOM_STATE
)

print("Train interactions:", train_df.shape)
print("Test interactions:", test_df.shape)

# ------------------------------------------------------------
# 9) BUILD SPARSE MATRICES (AUGMENTED)
# ------------------------------------------------------------
train_mat = build_sparse_matrix(train_df, n_users, n_items)
test_mat  = build_sparse_matrix(test_df,  n_users, n_items)

print("Sparse shapes — train:", train_mat.shape, "test:", test_mat.shape)

# ------------------------------------------------------------
# 7) EVALUATION METRICS (Recall@K, MAP@K)
# ------------------------------------------------------------
def recall_at_k(recommender_fn, train_mat, test_mat, k=10, n_users_eval=None):
    """
    recommender_fn: function(user_id, N) -> list/array of recommended item_ids
    """
    n_users, _ = train_mat.shape
    if n_users_eval is None or n_users_eval > n_users:
        n_users_eval = n_users

    recalls = []
    for u in range(n_users_eval):
        test_items = test_mat[u].indices
        if len(test_items) == 0:
            continue
        rec_items = recommender_fn(u, k)
        hits = len(set(test_items) & set(rec_items))
        recalls.append(hits / len(test_items))
    return float(np.mean(recalls)) if len(recalls) > 0 else np.nan

def map_at_k(recommender_fn, train_mat, test_mat, k=10, n_users_eval=None):
    n_users, _ = train_mat.shape
    if n_users_eval is None or n_users_eval > n_users:
        n_users_eval = n_users

    aps = []
    for u in range(n_users_eval):
        test_items = test_mat[u].indices
        if len(test_items) == 0:
            continue
        rec_items = recommender_fn(u, k)

        score = 0.0
        hits = 0
        for i, item in enumerate(rec_items, start=1):
            if item in test_items:
                hits += 1
                score += hits / i
        if hits > 0:
            aps.append(score / min(len(test_items), k))
        else:
            aps.append(0.0)
    return float(np.mean(aps)) if len(aps) > 0 else np.nan

# ------------------------------------------------------------
# 8) MODEL 1 — POPULARITY BASELINE
# ------------------------------------------------------------
print("\n=== MODEL 1: Popularity baseline ===")
item_popularity = np.asarray(train_mat.sum(axis=0)).ravel()
popular_items_sorted = np.argsort(-item_popularity)  # descending

def pop_recommender(user_id, N=10):
    # filter out items the user already interacted with
    known_items = set(train_mat[user_id].indices)
    rec = [i for i in popular_items_sorted if i not in known_items]
    return rec[:N]

pop_recall10 = recall_at_k(pop_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
pop_map10    = map_at_k(pop_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"Popularity → Recall@10 = {pop_recall10:.4f}, MAP@10 = {pop_map10:.4f}")

# ------------------------------------------------------------
# 9) MODEL 2 — Item-Item kNN (Collaborative)
# ------------------------------------------------------------
print("\n=== MODEL 2: Item-kNN (cosine) ===")


# item vectors = columns of train_mat  → (n_items x n_users)
item_user = train_mat.T.tocsr()
item_user_norm = normalize(item_user, axis=1)
# cosine similarity matrix between items → (n_items x n_items)
item_sim = (item_user_norm @ item_user_norm.T).tocsr()
item_sim.setdiag(0)

# For our tiny n_items, we can safely use a dense matrix
item_sim_dense = item_sim.toarray()

def itemknn_recommender(user_id, N=10):
    user_row = train_mat[user_id]              # (1 x n_items) sparse
    user_items = user_row.indices              # indices of items user has
    if len(user_items) == 0:
        # cold-start user → fall back to popularity
        return pop_recommender(user_id, N)

    # weights for the items this user has interacted with
    user_weights_full = np.asarray(user_row.toarray()).ravel()
    user_weights = user_weights_full[user_items]   # shape (|user_items|,)

    # scores for each candidate item:
    #   score(i) = sum_j sim(i, j) * weight_j  for j in user_items
    # item_sim_dense: (n_items x n_items)
    # take only columns of items user has
    scores = item_sim_dense[:, user_items] @ user_weights   # (n_items,)

    # don't recommend items the user already has
    scores[user_items] = -np.inf

    # top-N items by score
    rec_items = np.argsort(-scores)[:N]
    return rec_items


knn_recall10 = recall_at_k(itemknn_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
knn_map10    = map_at_k(itemknn_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"Item-kNN   → Recall@10 = {knn_recall10:.4f}, MAP@10 = {knn_map10:.4f}")

# ------------------------------------------------------------
# 10) MODEL 3 — ALS (Matrix Factorization with implicit)
# ------------------------------------------------------------
print("\n=== MODEL 3: ALS (implicit) ===")

als_model = implicit.als.AlternatingLeastSquares(
    factors=64,
    regularization=0.01,
    iterations=15,
    random_state=RANDOM_STATE
)

# implicit expects user_items (users x items)
user_item_mat = train_mat.tocsr()
als_model.fit(user_item_mat)
print("ALS training done.")

def als_recommender(user_id, N=10):
    user_items = user_item_mat[user_id]
    item_ids, scores = als_model.recommend(
        userid=user_id,
        user_items=user_items,
        N=N,
        filter_already_liked_items=True
    )
    return item_ids

als_recall10 = recall_at_k(als_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
als_map10    = map_at_k(als_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"ALS        → Recall@10 = {als_recall10:.4f}, MAP@10 = {als_map10:.4f}")

# ------------------------------------------------------------
# 11) MODEL COMPARISON SUMMARY
# ------------------------------------------------------------
summary = pd.DataFrame({
    "model": ["Popularity", "Item-kNN", "ALS"],
    "Recall@10": [pop_recall10, knn_recall10, als_recall10],
    "MAP@10":    [pop_map10,   knn_map10,   als_map10],
})
print("\n=== MODEL COMPARISON ===")
print(summary)

# ------------------------------------------------------------
# 12) EXAMPLE PERSONALIZED RECOMMENDATIONS
# ------------------------------------------------------------
print("\n=== EXAMPLE RECOMMENDATIONS (ALS) ===")
example_customer = tails_small['customer_id'].iloc[0]
raw_user_key = "cust_" + str(example_customer)

if raw_user_key in user_encoder.classes_:
    uid = user_encoder.transform([raw_user_key])[0]
    rec_items = als_recommender(uid, N=10)
    rec_keys  = item_encoder.inverse_transform(rec_items)

    rec_df = items_catalog[items_catalog['item_key'].isin(rec_keys)].copy()
    rec_df['rank'] = rec_df['item_key'].map({k: i for i, k in enumerate(rec_keys)})
    rec_df = rec_df.sort_values('rank').reset_index(drop=True)

    print(f"Recommendations for customer_id={example_customer}:")
    print(rec_df[['item_key', 'source', 'category', 'size']].head(10))
else:
    print(f"Customer {example_customer} not in training set.")


Loading CSVs...
tails    →  49042 rows ×  36 cols
store    →    879 rows ×  11 cols
chewy    →    175 rows ×  21 cols
amz      →    343 rows ×  15 cols
ali      →   1998 rows ×   5 cols
usst     →    875 rows ×  52 cols
metrics  →   2038 rows ×  82 cols

=== BASIC EDA ===

[TAILS] pet_food_customer_orders overview
            customer_id               pet_id  pet_order_number pet_food_tier  \
0  10574848487411271014  4466839344031767293                 2  superpremium   
1  10574848487411271014  4466839344031767293                 1  superpremium   
2  10574848487411271014  4466839344031767293                 8  superpremium   
3  10574848487411271014  4466839344031767293                 4  superpremium   
4  10574848487411271014  4466839344031767293                 9  superpremium   

  wet_tray_size  wet_trays  
0          150g         11  
1           NaN          0  
2          150g         60  
3          150g         26  
4          150g         60  

[TAILS] pet_food_tier value 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()



Encoded users: 3439, items: 159
Train interactions: (4375, 5)
Test interactions: (3439, 5)
Sparse shapes — train: (3439, 159) test: (3439, 159)

Unified item catalog size: (3557, 8)
                               item_key source      category  size pet_type  \
0  tails_superpremium|150g|mature|large  tails  superpremium  150g      NaN   
1  tails_superpremium|none|mature|large  tails  superpremium  none      NaN   
2      tails_premium|none|mature|medium  tails       premium  none      NaN   
3    tails_mid|none|half_maturity|small  tails           mid  none      NaN   
4          tails_mid|none|weaning|small  tails           mid  none      NaN   

  name brand  item_id  
0  NaN   NaN      109  
1  NaN   NaN      145  
2  NaN   NaN       90  
3  NaN   NaN       36  
4  NaN   NaN       51  
Core items: 1041 External items: 2516
Similarity matrix shape: (2516, 1041)
Number of core items with real users: 159
Synthetic interactions generated: 95
Original interactions: (7814, 5)
Augmented 

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

ALS training done.
ALS        → Recall@10 = 0.3605, MAP@10 = 0.2025

=== MODEL COMPARISON ===
        model  Recall@10    MAP@10
0  Popularity     0.1215  0.050592
1    Item-kNN     0.6930  0.365411
2         ALS     0.3605  0.202511

=== EXAMPLE RECOMMENDATIONS (ALS) ===
Recommendations for customer_id=10574848487411271014:
                                      item_key source      category  size
0         tails_superpremium|none|senior|large  tails  superpremium  none
1  tails_superpremium|150g|half_maturity|large  tails  superpremium  150g
2                 tails_mid|150g|senior|medium  tails           mid  150g
3       tails_premium|none|half_maturity|large  tails       premium  none
4  tails_superpremium|150g|half_maturity|giant  tails  superpremium  150g
5                  tails_mid|300g|mature|large  tails           mid  300g
6         tails_superpremium|150g|senior|large  tails  superpremium  150g
7                  tails_mid|300g|mature|small  tails           mid  300g
8      

In [4]:
# =========================================
# A) GRAPH-BASED MODEL — LightGCN style
# =========================================
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.sparse as sp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Build normalized adjacency A_hat for user–item graph
R = train_mat  # (n_users x n_items)
num_users, num_items = R.shape
num_nodes = num_users + num_items

# Adjacency: [[0, R],
#             [R^T, 0]]
upper = sp.hstack([sp.csr_matrix((num_users, num_users)), R])
lower = sp.hstack([R.T, sp.csr_matrix((num_items, num_items))])
A = sp.vstack([upper, lower]).tocsr()

# D^{-1/2} A D^{-1/2}
rowsum = np.array(A.sum(axis=1)).flatten()
d_inv_sqrt = np.power(rowsum, -0.5)
d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
D_inv_sqrt = sp.diags(d_inv_sqrt)
A_hat = D_inv_sqrt.dot(A).dot(D_inv_sqrt).tocsr()

# Convert to torch sparse
A_hat_coo = A_hat.tocoo()
indices = torch.from_numpy(
    np.vstack([A_hat_coo.row, A_hat_coo.col]).astype(np.int64)
)
values = torch.from_numpy(A_hat_coo.data.astype(np.float32))
A_hat_torch = torch.sparse_coo_tensor(
    indices, values, (num_nodes, num_nodes), device=device
).coalesce()

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32, num_layers=2):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def propagate(self):
        all_emb = torch.cat(
            [self.user_emb.weight, self.item_emb.weight], dim=0
        )  # (num_nodes, d)
        embs = [all_emb]

        for _ in range(self.num_layers):
            all_emb = torch.sparse.mm(A_hat_torch, all_emb)
            embs.append(all_emb)

        # Mean over layers
        embs = torch.stack(embs, dim=0).mean(dim=0)
        user_final = embs[:self.num_users]
        item_final = embs[self.num_users:]
        return user_final, item_final

    def forward(self, users, pos_items, neg_items):
        user_final, item_final = self.propagate()
        u = user_final[users]
        p = item_final[pos_items]
        n = item_final[neg_items]

        pos_scores = (u * p).sum(dim=-1)
        neg_scores = (u * n).sum(dim=-1)

        # BPR loss + small L2 regularization
        loss = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8).mean()
        reg = (u.norm(2).pow(2) + p.norm(2).pow(2) + n.norm(2).pow(2)) / users.size(0)
        return loss + 1e-4 * reg

# Build training triples (u, i, j)
all_items = np.arange(num_items)

user_pos_items = {}
for uid, group in train_df.groupby('user_id'):
    user_pos_items[uid] = group['item_id'].values

def sample_batch(batch_size=1024):
    users = []
    pos = []
    neg = []
    for _ in range(batch_size):
        u = np.random.randint(0, num_users)
        pos_items = user_pos_items.get(u, None)
        if pos_items is None or len(pos_items) == 0:
            continue
        i = np.random.choice(pos_items)
        # sample negative item
        j = np.random.choice(all_items)
        while j in pos_items:
            j = np.random.choice(all_items)
        users.append(u)
        pos.append(i)
        neg.append(j)
    return (
        torch.tensor(users, dtype=torch.long, device=device),
        torch.tensor(pos,   dtype=torch.long, device=device),
        torch.tensor(neg,   dtype=torch.long, device=device),
    )

lightgcn = LightGCN(num_users, num_items, embedding_dim=32, num_layers=2).to(device)
optimizer = optim.Adam(lightgcn.parameters(), lr=1e-3)

print("\nTraining LightGCN...")
for epoch in range(5):  # small number of epochs for demo
    lightgcn.train()
    users_b, pos_b, neg_b = sample_batch(batch_size=2048)
    if users_b.numel() == 0:
        continue
    loss = lightgcn(users_b, pos_b, neg_b)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/5 - loss: {loss.item():.4f}")

lightgcn.eval()
with torch.no_grad():
    lg_user_emb, lg_item_emb = lightgcn.propagate()
lg_user_emb = lg_user_emb.cpu().numpy()
lg_item_emb = lg_item_emb.cpu().numpy()

def lightgcn_recommender(user_id, N=10):
    scores = lg_item_emb @ lg_user_emb[user_id]  # (n_items,)
    known = train_mat[user_id].indices
    scores[known] = -np.inf
    return np.argsort(-scores)[:N]

lg_recall10 = recall_at_k(lightgcn_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
lg_map10    = map_at_k(lightgcn_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"\nLightGCN  → Recall@10 = {lg_recall10:.4f}, MAP@10 = {lg_map10:.4f}")


Using device: cpu


/tmp/ipykernel_48/3835960981.py:25: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5)



Training LightGCN...
Epoch 1/5 - loss: 0.6931
Epoch 2/5 - loss: 0.6930
Epoch 3/5 - loss: 0.6929
Epoch 4/5 - loss: 0.6928
Epoch 5/5 - loss: 0.6926

LightGCN  → Recall@10 = 0.2050, MAP@10 = 0.0792


In [5]:
# =========================================
# B) HYBRID CF + CONTENT-BASED MODEL
# =========================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize as skl_normalize

# 1) Make sure items_catalog has item_id aligned with CURRENT item_encoder
items_catalog['item_id'] = -1
mask_known_items = items_catalog['item_key'].isin(item_encoder.classes_)
items_catalog.loc[mask_known_items, 'item_id'] = item_encoder.transform(
    items_catalog.loc[mask_known_items, 'item_key']
)

# Keep only items that actually exist in the user–item matrix
catalog_for_matrix = items_catalog[items_catalog['item_id'] >= 0].copy()

# Sort by item_id and ensure row index matches item_id
catalog_for_matrix = catalog_for_matrix.sort_values('item_id')
# texts indexed by item_id: index 0..n_items-1
texts = catalog_for_matrix.set_index('item_id').loc[range(n_items), :]['text' if 'text' in catalog_for_matrix.columns else None]

# If you didn't build "text" before, build it now:
if 'text' not in catalog_for_matrix.columns:
    def make_item_text(row):
        parts = [
            str(row.get('source', '')),
            str(row.get('category', '')),
            str(row.get('size', '')),
            str(row.get('pet_type', '')),
            str(row.get('name', '')),
            str(row.get('brand', '')),
        ]
        return " ".join([p for p in parts if p and p != 'nan'])
    catalog_for_matrix['text'] = catalog_for_matrix.apply(make_item_text, axis=1)
    texts = catalog_for_matrix.set_index('item_id').loc[range(n_items), 'text']

# 2) TF–IDF for ALL items in matrix (rows = item_id)
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(texts.fillna(''))
X_tfidf = skl_normalize(X_tfidf)   # shape: (n_items, d)
assert X_tfidf.shape[0] == n_items, "TF-IDF rows must match n_items"

# 3) Hybrid recommender: alpha * ALS + (1-alpha) * content-based
def hybrid_recommender(user_id, alpha=0.6, N=10):
    user_row = train_mat[user_id]
    user_items = user_row.indices

    if len(user_items) == 0:
        # cold-start → popularity
        return pop_recommender(user_id, N)

    # (a) CF scores from ALS
    cf_scores = als_model.item_factors @ als_model.user_factors[user_id]  # (n_items,)

    # (b) Content-based profile from TF–IDF
    weights_full = np.asarray(user_row.toarray()).ravel()
    w = weights_full[user_items]
    if w.sum() == 0:
        w = np.ones_like(w)

    # profile_vec in TF–IDF space
    profile_vec = (X_tfidf[user_items].T @ w).ravel()
    norm = np.linalg.norm(profile_vec)
    if norm > 0:
        profile_vec = profile_vec / norm

    cb_scores = (X_tfidf @ profile_vec).ravel()  # (n_items,)

    # (c) Combine scores (same length)
    scores = alpha * cf_scores + (1.0 - alpha) * cb_scores

    # do not recommend already seen items
    scores[user_items] = -np.inf

    return np.argsort(-scores)[:N]

# 4) Evaluate hybrid model
hybrid_recall10 = recall_at_k(hybrid_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
hybrid_map10    = map_at_k(hybrid_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"\nHybrid (ALS+Content) → Recall@10 = {hybrid_recall10:.4f}, MAP@10 = {hybrid_map10:.4f}")

# 5) Example hybrid recommendations for same customer
print("\nHybrid recommendations for customer_id=", example_customer)
if raw_user_key in user_encoder.classes_:
    uid = user_encoder.transform([raw_user_key])[0]
    rec_items = hybrid_recommender(uid, N=10)
    rec_keys  = item_encoder.inverse_transform(rec_items)
    rec_df = items_catalog[items_catalog['item_key'].isin(rec_keys)].copy()
    rec_df['rank'] = rec_df['item_key'].map({k: i for i, k in enumerate(rec_keys)})
    rec_df = rec_df.sort_values('rank').reset_index(drop=True)
    print(rec_df[['item_key', 'source', 'category', 'size']].head(10))
else:
    print(f"Customer {example_customer} not in training set.")



Hybrid (ALS+Content) → Recall@10 = 0.0145, MAP@10 = 0.0084

Hybrid recommendations for customer_id= 10574848487411271014
                                       item_key source      category  size
0          tails_superpremium|none|senior|large  tails  superpremium  none
1   tails_superpremium|none|half_maturity|giant  tails  superpremium  none
2         tails_superpremium|none|weaning|large  tails  superpremium  none
3           tails_superpremium|none|weaning|toy  tails  superpremium  none
4     tails_superpremium|none|half_maturity|toy  tails  superpremium  none
5   tails_superpremium|none|half_maturity|large  tails  superpremium  none
6          tails_superpremium|none|mature|small  tails  superpremium  none
7  tails_superpremium|none|half_maturity|medium  tails  superpremium  none
8         tails_superpremium|none|mature|medium  tails  superpremium  none
9            tails_superpremium|none|mature|toy  tails  superpremium  none


In [6]:
# =========================================
# C) CVAE MODEL (user → latent → reconstructed items)
# =========================================
from torch.utils.data import Dataset, DataLoader

class UserItemDataset(Dataset):
    def __init__(self, mat):
        self.mat = mat

    def __len__(self):
        return self.mat.shape[0]

    def __getitem__(self, idx):
        row = np.asarray(self.mat[idx].toarray()).ravel().astype('float32')
        row[row > 0] = 1.0  # binarize
        return torch.from_numpy(row)

train_dataset = UserItemDataset(train_mat)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

class CVAE(nn.Module):
    def __init__(self, n_items, hidden_dim=32, latent_dim=16):
        super().__init__()
        self.fc1 = nn.Linear(n_items, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc_dec1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, n_items)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc_dec1(z))
        return torch.sigmoid(self.fc_out(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

def vae_loss(x, x_recon, mu, logvar):
    # BCE reconstruction
    bce = nn.functional.binary_cross_entropy(x_recon, x, reduction='sum')
    # KL divergence
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return (bce + kld) / x.size(0)

cvae = CVAE(n_items, hidden_dim=32, latent_dim=16).to(device)
optimizer_cvae = optim.Adam(cvae.parameters(), lr=1e-3)

print("\nTraining CVAE...")
for epoch in range(10):
    cvae.train()
    epoch_loss = 0.0
    for batch in train_loader:
        batch = batch.to(device)
        x_recon, mu, logvar = cvae(batch)
        loss = vae_loss(batch, x_recon, mu, logvar)
        optimizer_cvae.zero_grad()
        loss.backward()
        optimizer_cvae.step()
        epoch_loss += loss.item() * batch.size(0)
    epoch_loss /= len(train_dataset)
    print(f"Epoch {epoch+1}/10 - loss: {epoch_loss:.4f}")

cvae.eval()

def cvae_recommender(user_id, N=10):
    row = np.asarray(train_mat[user_id].toarray()).ravel().astype('float32')
    x = torch.from_numpy(row).unsqueeze(0).to(device)
    with torch.no_grad():
        x_recon, _, _ = cvae(x)
    scores = x_recon.cpu().numpy().ravel()
    known = train_mat[user_id].indices
    scores[known] = -np.inf
    return np.argsort(-scores)[:N]

cvae_recall10 = recall_at_k(cvae_recommender, train_mat, test_mat, k=10, n_users_eval=2000)
cvae_map10    = map_at_k(cvae_recommender,    train_mat, test_mat, k=10, n_users_eval=2000)
print(f"\nCVAE      → Recall@10 = {cvae_recall10:.4f}, MAP@10 = {cvae_map10:.4f}")



Training CVAE...
Epoch 1/10 - loss: 111.0272
Epoch 2/10 - loss: 100.9457
Epoch 3/10 - loss: 89.3288
Epoch 4/10 - loss: 74.9731
Epoch 5/10 - loss: 55.6572
Epoch 6/10 - loss: 35.2388
Epoch 7/10 - loss: 22.8565
Epoch 8/10 - loss: 18.3884
Epoch 9/10 - loss: 16.6303
Epoch 10/10 - loss: 15.5306

CVAE      → Recall@10 = 0.2685, MAP@10 = 0.0985


In [7]:
all_models_summary = pd.DataFrame({
    "model": [
        "Popularity",
        "Item-kNN",
        "ALS",
        "LightGCN",
        "Hybrid ALS+Content",
        "CVAE"
    ],
    "Recall@10": [
        pop_recall10,
        knn_recall10,
        als_recall10,
        lg_recall10,
        hybrid_recall10,
        cvae_recall10
    ],
    "MAP@10": [
        pop_map10,
        knn_map10,
        als_map10,
        lg_map10,
        hybrid_map10,
        cvae_map10
    ]
})
print("\n=== ALL MODELS COMPARISON ===")
print(all_models_summary)



=== ALL MODELS COMPARISON ===
                model  Recall@10    MAP@10
0          Popularity     0.1215  0.050592
1            Item-kNN     0.6930  0.365411
2                 ALS     0.3605  0.202511
3            LightGCN     0.2050  0.079232
4  Hybrid ALS+Content     0.0145  0.008391
5                CVAE     0.2685  0.098537


In [8]:
# Add a simple combined score = Recall@10 × MAP@10
all_models_summary["Score_product"] = (
    all_models_summary["Recall@10"] * all_models_summary["MAP@10"]
)

print("\n=== ALL MODELS COMPARISON (with simple product score) ===")
print(all_models_summary)



=== ALL MODELS COMPARISON (with simple product score) ===
                model  Recall@10    MAP@10  Score_product
0          Popularity     0.1215  0.050592       0.006147
1            Item-kNN     0.6930  0.365411       0.253230
2                 ALS     0.3605  0.202511       0.073005
3            LightGCN     0.2050  0.079232       0.016243
4  Hybrid ALS+Content     0.0145  0.008391       0.000122
5                CVAE     0.2685  0.098537       0.026457


In [9]:
import time

def timed_eval(model_name, recommender_fn, k=10, n_users_eval=2000):
    start = time.perf_counter()
    rec = recall_at_k(recommender_fn, train_mat, test_mat, k=k, n_users_eval=n_users_eval)
    m   = map_at_k(recommender_fn,    train_mat, test_mat, k=k, n_users_eval=n_users_eval)
    elapsed = time.perf_counter() - start
    print(f"{model_name}: Recall@{k}={rec:.4f}, MAP@{k}={m:.4f}, "
          f"product={rec*m:.4f}, time={elapsed:.3f}s")
    return rec, m, elapsed


In [10]:
import time
import numpy as np

# Helper: time recall@k + MAP@k for a given recommender
def timed_eval(model_name, recommender_fn, k=10, n_users_eval=2000):
    start = time.perf_counter()
    rec = recall_at_k(recommender_fn, train_mat, test_mat,
                      k=k, n_users_eval=n_users_eval)
    m   = map_at_k(recommender_fn,    train_mat, test_mat,
                   k=k, n_users_eval=n_users_eval)
    elapsed = time.perf_counter() - start
    score_prod = rec * m
    print(f"{model_name:18s}: "
          f"Recall@{k}={rec:.4f}, "
          f"MAP@{k}={m:.4f}, "
          f"product={score_prod:.44f}, "
          f"time={elapsed:.4f}s")
    return rec, m, elapsed

print("\n=== TIMED EVALUATION FOR ALL MODELS ===")

# 1) Popularity
pop_rec_t, pop_map_t, pop_time = timed_eval("Popularity", pop_recommender)

# 2) Item-kNN
knn_rec_t, knn_map_t, knn_time = timed_eval("Item-kNN", itemknn_recommender)

# 3) ALS
als_rec_t, als_map_t, als_time = timed_eval("ALS", als_recommender)

# 4) LightGCN
lg_rec_t, lg_map_t, lg_time = timed_eval("LightGCN", lightgcn_recommender)

# 5) Hybrid ALS + Content
hyb_rec_t, hyb_map_t, hyb_time = timed_eval("Hybrid ALS+Content", hybrid_recommender)

# 6) CVAE
cvae_rec_t, cvae_map_t, cvae_time = timed_eval("CVAE", cvae_recommender)

# Build a full summary table
timed_summary = pd.DataFrame({
    "model": [
        "Popularity",
        "Item-kNN",
        "ALS",
        "LightGCN",
        "Hybrid ALS+Content",
        "CVAE"
    ],
    "Recall@10": [
        pop_rec_t,
        knn_rec_t,
        als_rec_t,
        lg_rec_t,
        hyb_rec_t,
        cvae_rec_t
    ],
    "MAP@10": [
        pop_map_t,
        knn_map_t,
        als_map_t,
        lg_map_t,
        hyb_map_t,
        cvae_map_t
    ],
    "Score_product": [
        pop_rec_t * pop_map_t,
        knn_rec_t * knn_map_t,
        als_rec_t * als_map_t,
        lg_rec_t * lg_map_t,
        hyb_rec_t * hyb_map_t,
        cvae_rec_t * cvae_map_t
    ],
    "Time_s": [
        pop_time,
        knn_time,
        als_time,
        lg_time,
        hyb_time,
        cvae_time
    ]
})

print("\n=== ALL MODELS (WITH TIME) ===")
print(timed_summary)



=== TIMED EVALUATION FOR ALL MODELS ===
Popularity        : Recall@10=0.1215, MAP@10=0.0506, product=0.00614693571428571311487942807616491336375475, time=0.7734s
Item-kNN          : Recall@10=0.6930, MAP@10=0.3654, product=0.25322976249999995523864981805672869086265564, time=0.7812s
ALS               : Recall@10=0.3605, MAP@10=0.2025, product=0.07300525555555555101694409358970005996525288, time=1.1267s
LightGCN          : Recall@10=0.2050, MAP@10=0.0792, product=0.01624262996031745898140030703871161676943302, time=0.7796s
Hybrid ALS+Content: Recall@10=0.0145, MAP@10=0.0084, product=0.00012166478174603175173035946032840115549334, time=2.4976s
CVAE              : Recall@10=0.2550, MAP@10=0.1001, product=0.02552620833333333824621824703626771224662662, time=2.2941s

=== ALL MODELS (WITH TIME) ===
                model  Recall@10    MAP@10  Score_product    Time_s
0          Popularity     0.1215  0.050592       0.006147  0.773442
1            Item-kNN     0.6930  0.365411       0.253230  

In [11]:
import pickle
import joblib
import numpy as np
import scipy.sparse as sp
import os

os.makedirs("models", exist_ok=True)

# 1) Save encoders
with open("models/user_encoder.pkl", "wb") as f:
    pickle.dump(user_encoder, f)

with open("models/item_encoder.pkl", "wb") as f:
    pickle.dump(item_encoder, f)

# 2) Save item catalog (for showing product info on website)
items_catalog.to_parquet("models/items_catalog.parquet")

# 3) Save interaction matrices (optional but useful)
sp.save_npz("models/train_mat.npz", train_mat)
sp.save_npz("models/test_mat.npz",  test_mat)

# 4) Save Popularity model (just item_popularity and sorted indices)
np.save("models/item_popularity.npy", item_popularity)
np.save("models/popular_items_sorted.npy", popular_items_sorted)

# 5) Save Item-kNN similarity matrix
np.save("models/item_sim_dense.npy", item_sim_dense)

# 6) Save ALS model (entire object via pickle)
with open("models/als_model.pkl", "wb") as f:
    pickle.dump(als_model, f)

# 7) Save Hybrid content components (TF-IDF vectorizer + X_tfidf)
with open("models/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

sp.save_npz("models/X_tfidf.npz", X_tfidf)

print("✅ All classical models & artifacts saved in ./models/")


✅ All classical models & artifacts saved in ./models/


In [12]:
# serve_recommender.py
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp

# ---- Load artifacts ----
with open("models/user_encoder.pkl", "rb") as f:
    user_encoder = pickle.load(f)

with open("models/item_encoder.pkl", "rb") as f:
    item_encoder = pickle.load(f)

items_catalog = pd.read_parquet("models/items_catalog.parquet")

train_mat = sp.load_npz("models/train_mat.npz")

item_popularity      = np.load("models/item_popularity.npy")
popular_items_sorted = np.load("models/popular_items_sorted.npy")
item_sim_dense       = np.load("models/item_sim_dense.npy")

with open("models/als_model.pkl", "rb") as f:
    als_model = pickle.load(f)

with open("models/tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

X_tfidf = sp.load_npz("models/X_tfidf.npz")
n_users, n_items = train_mat.shape


In [13]:
def pop_recommender(user_id, N=10):
    known_items = set(train_mat[user_id].indices)
    rec = [i for i in popular_items_sorted if i not in known_items]
    return np.array(rec[:N], dtype=int)

def itemknn_recommender(user_id, N=10):
    user_row = train_mat[user_id]
    user_items = user_row.indices
    if len(user_items) == 0:
        return pop_recommender(user_id, N)

    user_weights_full = np.asarray(user_row.toarray()).ravel()
    user_weights = user_weights_full[user_items]

    scores = item_sim_dense[:, user_items] @ user_weights
    scores[user_items] = -np.inf

    rec_items = np.argsort(-scores)[:N]
    return rec_items

def als_recommender(user_id, N=10):
    user_items = train_mat[user_id]
    item_ids, scores = als_model.recommend(
        userid=user_id,
        user_items=user_items,
        N=N,
        filter_already_liked_items=True
    )
    return item_ids

# Hybrid: ALS + content
def hybrid_recommender(user_id, alpha=0.6, N=10):
    user_row = train_mat[user_id]
    user_items = user_row.indices
    if len(user_items) == 0:
        return pop_recommender(user_id, N)

    # CF scores
    cf_scores = als_model.item_factors @ als_model.user_factors[user_id]

    # Content scores from TF-IDF profile
    weights_full = np.asarray(user_row.toarray()).ravel()
    w = weights_full[user_items]
    if w.sum() == 0:
        w = np.ones_like(w)

    profile_vec = (X_tfidf[user_items].T @ w).ravel()
    norm = np.linalg.norm(profile_vec)
    if norm > 0:
        profile_vec = profile_vec / norm

    cb_scores = (X_tfidf @ profile_vec).ravel()

    scores = alpha * cf_scores + (1 - alpha) * cb_scores
    scores[user_items] = -np.inf

    rec_items = np.argsort(-scores)[:N]
    return rec_items


In [14]:
MODEL_MAP = {
    "popularity": pop_recommender,
    "itemknn":    itemknn_recommender,
    "als":        als_recommender,
    "hybrid":     hybrid_recommender,
    "lightgcn": lightgcn_recommender,
    "cvae":     cvae_recommender,
}

def recommend_for_customer(customer_id: int, model_name: str = "hybrid", N: int = 10):
    """
    Input:
      customer_id: the raw Tails customer_id from your database (int or str)
      model_name: one of 'popularity', 'itemknn', 'als', 'hybrid', ...
      N: number of items to recommend

    Output:
      List[dict] — each dict is one recommended item:
      {
         "item_key": "tails_superpremium|150g|mature|small",
         "source": "tails",
         "category": "superpremium",
         "size": "150g",
         "name": "...",
         "brand": "...",
         "rank": 0
      }
    """
    raw_user_key = f"cust_{customer_id}"

    # Cold-start: user not seen in training
    if raw_user_key not in user_encoder.classes_:
        # You can return top-N popular items directly here
        uid = None
        rec_ids = popular_items_sorted[:N]
    else:
        uid = user_encoder.transform([raw_user_key])[0]

        if model_name not in MODEL_MAP:
            model_name = "hybrid"  # default

        rec_fn = MODEL_MAP[model_name]
        # Some recommenders accept only (user_id, N), hybrid takes (user_id, N) via default alpha
        rec_ids = rec_fn(uid, N)

    # Map item_ids back to item_keys
    rec_ids = np.array(rec_ids, dtype=int)
    rec_keys = item_encoder.inverse_transform(rec_ids)

    # Enrich with catalog info
    rec_df = items_catalog[items_catalog["item_key"].isin(rec_keys)].copy()
    rank_map = {k: i for i, k in enumerate(rec_keys)}
    rec_df["rank"] = rec_df["item_key"].map(rank_map)

    rec_df = rec_df.sort_values("rank")

    # Choose what fields you want your website to see:
    return rec_df[[
        "item_key", "source", "category", "size", "name", "brand", "rank"
    ]].to_dict(orient="records")


In [15]:
# Example local test:
if __name__ == "__main__":
    example_customer = 10574848487411271014
    recs = recommend_for_customer(example_customer, model_name="hybrid", N=10)
    for r in recs:
        print(r)


{'item_key': 'tails_mid|150g|senior|medium', 'source': 'tails', 'category': 'mid', 'size': '150g', 'name': None, 'brand': None, 'rank': 0}
{'item_key': 'tails_mid|300g|mature|large', 'source': 'tails', 'category': 'mid', 'size': '300g', 'name': None, 'brand': None, 'rank': 1}
{'item_key': 'tails_mid|300g|mature|small', 'source': 'tails', 'category': 'mid', 'size': '300g', 'name': None, 'brand': None, 'rank': 2}
{'item_key': 'tails_premium|150g|half_maturity|medium', 'source': 'tails', 'category': 'premium', 'size': '150g', 'name': None, 'brand': None, 'rank': 3}
{'item_key': 'tails_mid|150g|weaning|large', 'source': 'tails', 'category': 'mid', 'size': '150g', 'name': None, 'brand': None, 'rank': 4}
{'item_key': 'tails_mid|300g|senior|medium', 'source': 'tails', 'category': 'mid', 'size': '300g', 'name': None, 'brand': None, 'rank': 5}
{'item_key': 'tails_premium|150g|senior|small', 'source': 'tails', 'category': 'premium', 'size': '150g', 'name': None, 'brand': None, 'rank': 6}
{'item_

In [16]:
# ============================================================
# SESSION-BASED (CLICK / CART / PURCHASE) RECOMMENDER
# ============================================================
import numpy as np

# Event weights: tune these as you like
ACTION_WEIGHTS = {
    "view":        1.0,   # or "click"
    "click":       1.0,
    "add_to_cart": 2.0,
    "purchase":    3.0,
}

def _item_ids_from_keys(item_keys):
    """
    Map list of item_key strings -> numpy array of item_ids.
    Unknown keys are silently dropped.
    """
    valid_keys = [k for k in item_keys if k in item_encoder.classes_]
    if not valid_keys:
        return np.array([], dtype=int)
    return item_encoder.transform(valid_keys)

def session_itemknn_scores(session_events):
    """
    session_events: list of dicts like
       {"item_key": "...", "event_type": "click" | "add_to_cart" | "purchase"}

    Returns:
        scores : np.array shape (n_items,) — score for each item_id
        seen_ids : list of item_ids present in session
    """
    scores = np.zeros(item_sim_dense.shape[0], dtype=float)
    seen_ids = []

    for ev in session_events:
        key = ev.get("item_key")
        et  = ev.get("event_type", "click")
        w   = ACTION_WEIGHTS.get(et, 1.0)

        if key not in item_encoder.classes_:
            continue

        iid = item_encoder.transform([key])[0]
        seen_ids.append(iid)

        # Add similarity vector for this item, weighted by action type
        scores += w * item_sim_dense[:, iid]

    # Cold-start session: no valid items → fallback to popularity
    if not seen_ids:
        scores = np.zeros(item_sim_dense.shape[0], dtype=float)
        # simple decreasing scores by popularity ranking
        scores[popular_items_sorted] = np.linspace(1.0, 0.0, len(popular_items_sorted))
        return scores, []

    # Don't recommend items already present in session
    scores[seen_ids] = -np.inf
    return scores, seen_ids

def recommend_from_session(session_events, N=10):
    """
    High-level API for your website: given current session actions, recommend items.

    session_events example:
    session_events = [
        {"item_key": "tails_superpremium|150g|mature|small", "event_type": "view"},
        {"item_key": "tails_superpremium|300g|mature|small", "event_type": "add_to_cart"},
        {"item_key": "amz_B00ABCXYZ", "event_type": "purchase"},
    ]
    """
    scores, _ = session_itemknn_scores(session_events)
    rec_ids = np.argsort(-scores)[:N]          # top-N item_ids
    rec_keys = item_encoder.inverse_transform(rec_ids)

    rec_df = items_catalog[items_catalog["item_key"].isin(rec_keys)].copy()
    rank_map = {k: i for i, k in enumerate(rec_keys)}
    rec_df["rank"] = rec_df["item_key"].map(rank_map)
    rec_df = rec_df.sort_values("rank").reset_index(drop=True)

    return rec_df[[
        "item_key", "source", "category", "size", "name", "brand", "rank"
    ]].to_dict(orient="records")

# ----------------- quick demo -----------------
# Example session:
example_session = [
    {"item_key": "tails_superpremium|150g|mature|small", "event_type": "view"},
    {"item_key": "tails_superpremium|300g|mature|small", "event_type": "add_to_cart"},
]

print("\n=== SESSION-BASED RECS (example) ===")
for rec in recommend_from_session(example_session, N=5):
    print(rec)



=== SESSION-BASED RECS (example) ===
{'item_key': 'tails_superpremium|150g|senior|small', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 0}
{'item_key': 'tails_superpremium|300g|half_maturity|small', 'source': 'tails', 'category': 'superpremium', 'size': '300g', 'name': None, 'brand': None, 'rank': 1}
{'item_key': 'tails_mid|300g|senior|large', 'source': 'tails', 'category': 'mid', 'size': '300g', 'name': None, 'brand': None, 'rank': 2}
{'item_key': 'tails_superpremium|none|mature|small', 'source': 'tails', 'category': 'superpremium', 'size': 'none', 'name': None, 'brand': None, 'rank': 3}
{'item_key': 'tails_superpremium|none|half_maturity|small', 'source': 'tails', 'category': 'superpremium', 'size': 'none', 'name': None, 'brand': None, 'rank': 4}


In [17]:
# ============================================================
# 2.2 SESSION-BASED & TEXT-BASED RECOMMENDERS
#    - Session / click-based recs from item(s)
#    - Text → items (single description)
#    - Two descriptions combined (profile + intent)
# ============================================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize as skl_normalize

# ------------------------------------------------------------
# A) Build item text features (for content similarity)
# ------------------------------------------------------------
# Use only items that appear in the user–item matrix (item_id >= 0)
known_items = items_catalog[items_catalog["item_id"] >= 0].copy()
known_items = known_items.sort_values("item_id")
known_ids   = known_items["item_id"].to_numpy()

# Build text for each item (source + category + size + pet_type + name + brand)
def make_item_text(row):
    parts = [
        str(row.get("source", "")),
        str(row.get("category", "")),
        str(row.get("size", "")),
        str(row.get("pet_type", "")),
        str(row.get("name", "")),
        str(row.get("brand", "")),
    ]
    parts = [p for p in parts if p and p != "nan"]
    return " ".join(parts)

known_items["text"] = known_items.apply(make_item_text, axis=1).fillna("")

# TF–IDF over item texts
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf    = vectorizer.fit_transform(known_items["text"])
X_tfidf    = skl_normalize(X_tfidf)        # (n_known_items, d), L2-normalized

# ------------------------------------------------------------
# B) Small helper: from query vector → scores over known items
# ------------------------------------------------------------
def _scores_from_query_vec(q_vec):
    """
    q_vec: 1×d sparse vector (already normalized)
    Returns a dense 1D array of similarity scores for each known item.
    """
    sims   = X_tfidf @ q_vec.T        # csr_matrix (n_known_items × 1)
    scores = np.asarray(sims.toarray()).ravel()
    return scores

# ------------------------------------------------------------
# C) SESSION / CLICK-BASED RECOMMENDER
#    "If the user just clicked item X, recommend similar items"
# ------------------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize as skl_normalize
import scipy.sparse as sp

# ... (known_items, vectorizer, X_tfidf, _scores_from_query_vec as before)

def session_recommender_from_clicks(clicked_item_keys, N=10):
    """
    clicked_item_keys: string or list of item_key(s) that user just interacted with
    Returns: list of dicts with recommended items.
    """
    # Accept single string or list
    if isinstance(clicked_item_keys, str):
        clicked_item_keys = [clicked_item_keys]

    # Map item_key -> item_id, keep only items that exist in matrix
    valid_item_ids = []
    for k in clicked_item_keys:
        if k in item_encoder.classes_:
            iid = item_encoder.transform([k])[0]
            valid_item_ids.append(iid)

    if not valid_item_ids:
        # Fall back to popularity if we cannot map any clicked items
        print("[session] No valid clicked items found, fallback to popularity.")
        rec_items = pop_recommender(0, N)
        rec_keys  = item_encoder.inverse_transform(rec_items)
    else:
        # Use the TF–IDF content similarity of these clicked items
        rows = [iid for iid in valid_item_ids if iid < X_tfidf.shape[0]]
        if not rows:
            print("[session] Item IDs not present in known_items, fallback to popularity.")
            rec_items = pop_recommender(0, N)
            rec_keys  = item_encoder.inverse_transform(rec_items)
        else:
            # ---- FIXED PART ----
            # sum() returns a numpy.matrix; turn it back into a CSR sparse matrix
            q_vec = X_tfidf[rows].sum(axis=0)          # 1×d numpy.matrix
            q_vec = sp.csr_matrix(q_vec)               # → 1×d csr_matrix
            q_vec = skl_normalize(q_vec, norm="l2")    # L2 norm = 1
            # --------------------

            scores = _scores_from_query_vec(q_vec)

            # Do not recommend clicked items themselves
            for iid in valid_item_ids:
                if iid < len(scores):
                    scores[iid] = -np.inf

            top_idx      = np.argsort(-scores)[:N]
            top_item_ids = known_items.iloc[top_idx]["item_id"].to_numpy()
            rec_keys     = item_encoder.inverse_transform(top_item_ids)

    rec_df   = items_catalog[items_catalog["item_key"].isin(rec_keys)].copy()
    rank_map = {k: i for i, k in enumerate(rec_keys)}
    rec_df["rank"] = rec_df["item_key"].map(rank_map)
    rec_df = rec_df.sort_values("rank").reset_index(drop=True)

    return rec_df[["item_key", "source", "category", "size", "name", "brand", "rank"]] \
        .to_dict(orient="records")


# ------------------------------------------------------------
# D) TEXT-BASED RECOMMENDER (single description)
# ------------------------------------------------------------
def recommend_from_text(description, N=10):
    """
    Recommend items purely from a user text description.

    Example:
        recs = recommend_from_text(
            "dry food for small senior dog, sensitive stomach, grain free",
            N=10
        )
    """
    q = vectorizer.transform([description])
    q = skl_normalize(q)   # 1×d

    scores       = _scores_from_query_vec(q)
    top_idx      = np.argsort(-scores)[:N]
    top_item_ids = known_items.iloc[top_idx]["item_id"].to_numpy()

    rec_keys = item_encoder.inverse_transform(top_item_ids)
    rec_df   = items_catalog[items_catalog["item_key"].isin(rec_keys)].copy()
    rank_map = {k: i for i, k in enumerate(rec_keys)}
    rec_df["rank"] = rec_df["item_key"].map(rank_map)
    rec_df = rec_df.sort_values("rank").reset_index(drop=True)

    return rec_df[["item_key", "source", "category", "size", "name", "brand", "rank"]] \
        .to_dict(orient="records")

# ------------------------------------------------------------
# E) TEXT-BASED RECOMMENDER (two descriptions combined)
# ------------------------------------------------------------
def recommend_from_two_texts(desc1, desc2, w1=0.5, w2=0.5, N=10):
    """
    Combine two descriptions (e.g. 'long-term profile' + 'current need').

    Example:
        desc_profile = "small senior dog, sensitive stomach"
        desc_intent  = "wet food with chicken and rice"
        recs = recommend_from_two_texts(desc_profile, desc_intent, w1=0.7, w2=0.3, N=10)
    """
    q1 = vectorizer.transform([desc1])
    q2 = vectorizer.transform([desc2])

    q1 = skl_normalize(q1)
    q2 = skl_normalize(q2)

    # Weighted sum of two queries, then normalize again
    q_comb = w1 * q1 + w2 * q2
    q_comb = skl_normalize(q_comb)

    scores       = _scores_from_query_vec(q_comb)
    top_idx      = np.argsort(-scores)[:N]
    top_item_ids = known_items.iloc[top_idx]["item_id"].to_numpy()

    rec_keys = item_encoder.inverse_transform(top_item_ids)
    rec_df   = items_catalog[items_catalog["item_key"].isin(rec_keys)].copy()
    rank_map = {k: i for i, k in enumerate(rec_keys)}
    rec_df["rank"] = rec_df["item_key"].map(rank_map)
    rec_df = rec_df.sort_values("rank").reset_index(drop=True)

    return rec_df[["item_key", "source", "category", "size", "name", "brand", "rank"]] \
        .to_dict(orient="records")

# ------------------------------------------------------------
# F) QUICK DEMOS (you can comment these out in production)
# ------------------------------------------------------------
print("\n=== DEMO: SESSION-BASED RECS (from one clicked item) ===")
demo_clicked = "tails_superpremium|150g|mature|small"
for rec in session_recommender_from_clicks(demo_clicked, N=5):
    print(rec)

print("\n=== DEMO: TEXT-BASED RECS (single description) ===")
demo_desc = "dry cat food for indoor adult cat, high protein, hairball control"
for rec in recommend_from_text(demo_desc, N=5):
    print(rec)

print("\n=== DEMO: TEXT-BASED RECS (two descriptions combined) ===")
desc_profile = "small senior dog, sensitive stomach"
desc_intent  = "grain-free chicken dry food"
for rec in recommend_from_two_texts(desc_profile, desc_intent, w1=0.7, w2=0.3, N=5):
    print(rec)



=== DEMO: SESSION-BASED RECS (from one clicked item) ===
{'item_key': 'tails_superpremium|150g|weaning|small', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 0}
{'item_key': 'tails_superpremium|150g|weaning|medium', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 1}
{'item_key': 'tails_superpremium|150g|half_maturity|giant', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 2}
{'item_key': 'tails_superpremium|150g|half_maturity|large', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 3}
{'item_key': 'tails_superpremium|150g|half_maturity|small', 'source': 'tails', 'category': 'superpremium', 'size': '150g', 'name': None, 'brand': None, 'rank': 4}

=== DEMO: TEXT-BASED RECS (single description) ===
{'item_key': 'ali_125', 'source': 'aliexpress', 'category': 'AliExpressPetSupplies', '

In [18]:
import os
import json
import joblib
import numpy as np
import pandas as pd
import scipy.sparse as sp

# -------------------------------------------------
# 1) Choose best model from timed_summary
# -------------------------------------------------
# timed_summary must already exist, e.g.:
# timed_summary = pd.DataFrame({
#     "model": [...],
#     "Recall@10": [...],
#     "MAP@10": [...],
#     "Score_product": [...],
#     "Time_s": [...]
# })

# Sort: highest Score_product, and if tie, prefer lower Time_s
best_row = timed_summary.sort_values(
    ["Score_product", "Time_s"],
    ascending=[False, True]
).iloc[0]

best_name = best_row["model"]
print("Best model selected:", best_name)
print(best_row)

# -------------------------------------------------
# 2) Prepare output folder + metadata
# -------------------------------------------------
MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

meta = {
    "best_model": best_name,
    "timed_summary": timed_summary.to_dict(orient="records"),
    "n_users": int(train_mat.shape[0]),
    "n_items": int(train_mat.shape[1]),
}

with open(os.path.join(MODEL_DIR, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

# -------------------------------------------------
# 3) Export best CF model (ALS / Item-kNN / etc.)
# -------------------------------------------------
if best_name == "ALS":
    # Save ALS model + encoders
    joblib.dump(als_model,       os.path.join(MODEL_DIR, "als_model.pkl"))
    joblib.dump(user_encoder,    os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder,    os.path.join(MODEL_DIR, "item_encoder.pkl"))

    # Save mapping item_id -> item_key + metadata
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved ALS model and encoders to", MODEL_DIR)

elif best_name == "Item-kNN":
    # Save similarity matrix and popularity vector
    np.savez(
        os.path.join(MODEL_DIR, "itemknn_model.npz"),
        item_sim_dense=item_sim_dense,
        item_popularity=item_popularity
    )
    joblib.dump(user_encoder, os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder, os.path.join(MODEL_DIR, "item_encoder.pkl"))
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved Item-kNN model and encoders to", MODEL_DIR)

elif best_name == "Popularity":
    # Only need popularity + encoders
    np.savez(
        os.path.join(MODEL_DIR, "pop_model.npz"),
        item_popularity=item_popularity
    )
    joblib.dump(user_encoder, os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder, os.path.join(MODEL_DIR, "item_encoder.pkl"))
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved Popularity model and encoders to", MODEL_DIR)

elif best_name == "LightGCN":
    # Assuming you have a trained lightgcn_model (PyTorch)
    joblib.dump(lightgcn_model,  os.path.join(MODEL_DIR, "lightgcn_model.pkl"))
    joblib.dump(user_encoder,    os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder,    os.path.join(MODEL_DIR, "item_encoder.pkl"))
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved LightGCN model and encoders to", MODEL_DIR)

elif best_name == "Hybrid ALS+Content":
    # Hybrid uses ALS + TF-IDF; save ALS + TF-IDF stuff
    joblib.dump(als_model,       os.path.join(MODEL_DIR, "als_model.pkl"))
    joblib.dump(user_encoder,    os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder,    os.path.join(MODEL_DIR, "item_encoder.pkl"))
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved Hybrid (ALS+Content) core (ALS + encoders) to", MODEL_DIR)

elif best_name == "CVAE":
    # Assuming you have a trained CVAE model object
    joblib.dump(cvae_model,      os.path.join(MODEL_DIR, "cvae_model.pkl"))
    joblib.dump(user_encoder,    os.path.join(MODEL_DIR, "user_encoder.pkl"))
    joblib.dump(item_encoder,    os.path.join(MODEL_DIR, "item_encoder.pkl"))
    items_catalog.to_parquet(os.path.join(MODEL_DIR, "items_catalog.parquet"))

    print("Saved CVAE model and encoders to", MODEL_DIR)

else:
    print("Warning: best model name not recognized, only metadata exported.")

# -------------------------------------------------
# 4) Export content / TF-IDF objects
#    (for click-based & text-based recommendations)
# -------------------------------------------------
# These are useful even if the best CF model is ALS or kNN, because
# you also want:
#  - recommend_from_text(...)
#  - session_recommender_from_clicks(...)
# to work on your website.
try:
    from scipy.sparse import save_npz

    # X_tfidf: (n_known_items, d)
    save_npz(os.path.join(MODEL_DIR, "X_tfidf.npz"), X_tfidf)

    # vectorizer: TfidfVectorizer
    joblib.dump(vectorizer, os.path.join(MODEL_DIR, "tfidf_vectorizer.pkl"))

    # known_items dataframe (id ↔ index in X_tfidf)
    known_items.to_parquet(os.path.join(MODEL_DIR, "known_items.parquet"))

    print("Saved TF-IDF content model (vectorizer + X_tfidf + known_items) to", MODEL_DIR)

except NameError:
    print("TF-IDF objects (vectorizer / X_tfidf / known_items) not found in scope, skip saving.")


Best model selected: Item-kNN
model            Item-kNN
Recall@10           0.693
MAP@10           0.365411
Score_product     0.25323
Time_s            0.78116
Name: 1, dtype: object
Saved Item-kNN model and encoders to saved_models
Saved TF-IDF content model (vectorizer + X_tfidf + known_items) to saved_models


In [19]:
import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------------------------------------------------
# CONFIG: base folder where you already copied the CSVs
# -------------------------------------------------------------------
BASE = Path("/kaggle/working/excel_csv")

# -------------------------------------------------------------------
# Utility: safe numeric conversions & normalization
# -------------------------------------------------------------------
def to_numeric_series(s, regex_keep=r"[0-9\.,]+"):
    """
    Convert an object/string Series with currency / commas / text to float.
    Example: '4,066.00' -> 4066.00, '25,17' (EU) -> 25.17, '5 sold' -> 5.0
    """
    s = s.astype(str).str.extract(f"({regex_keep})", expand=False)
    s = s.str.replace(" ", "", regex=False)
    # Replace thousand separators / decimal commas
    s = s.str.replace(".", "", regex=False)
    s = s.str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")


def minmax_norm(x):
    """Min-max normalize a numeric Series to [0,1]. NaNs -> 0."""
    x = x.astype(float)
    xmin, xmax = np.nanmin(x), np.nanmax(x)
    if np.isfinite(xmin) and np.isfinite(xmax) and xmax > xmin:
        out = (x - xmin) / (xmax - xmin)
    else:
        out = pd.Series(np.zeros(len(x)), index=x.index, dtype=float)
    return out.fillna(0.0)


def log1p_norm(x):
    """log1p then min-max normalize (useful for counts like sales / reviews)."""
    return minmax_norm(np.log1p(x.astype(float)))


# -------------------------------------------------------------------
# 0) TAILS PET FOOD ORDERS → aggregate to "items"
#     Use pet_food_tier + wet_tray_size + pet_life_stage_at_order + pet_breed_size
# -------------------------------------------------------------------
def build_tails_items():
    path = BASE / "pet-food-customer-orders-online" / "pet_food_customer_orders.csv"
    df = pd.read_csv(path)

    # Define a product-like key similar to your CF pipeline
    df["wet_tray_size_clean"] = df["wet_tray_size"].fillna("none").astype(str)
    df["pet_life_stage_at_order"] = df["pet_life_stage_at_order"].fillna("unknown").astype(str)
    df["pet_breed_size"] = df["pet_breed_size"].fillna("unknown").astype(str)
    df["pet_food_tier"] = df["pet_food_tier"].fillna("unknown").astype(str)

    df["item_key"] = (
        "tails_" +
        df["pet_food_tier"].astype(str) + "|" +
        df["wet_tray_size_clean"].astype(str) + "|" +
        df["pet_life_stage_at_order"].astype(str) + "|" +
        df["pet_breed_size"].astype(str)
    )
    df["source"] = "tails"

    # Aggregate per item_key
    grp = df.groupby("item_key", as_index=False).agg(
        n_orders=("customer_id", "nunique"),        # number of distinct customers
        n_rows=("customer_id", "size"),            # total rows / orders
        total_wet_trays=("wet_trays", "sum"),
        mean_kcal=("total_order_kcal", "mean"),
        mean_discount=("wet_food_discount_percent", "mean")
    )

    # Normalize features
    orders_norm = log1p_norm(grp["n_orders"])
    rows_norm = log1p_norm(grp["n_rows"])
    wet_trays_norm = log1p_norm(grp["total_wet_trays"])
    # lower discount is slightly better -> negative
    discount_norm = minmax_norm(grp["mean_discount"].fillna(0.0))

    # Composite score for tails
    grp["score_tails"] = (
        0.4 * orders_norm +
        0.3 * wet_trays_norm +
        0.2 * rows_norm -
        0.1 * discount_norm
    )

    # Recover representative fields per item_key (first value)
    rep_cols = df.groupby("item_key").agg({
        "pet_food_tier": "first",
        "wet_tray_size_clean": "first",
        "pet_breed_size": "first",
        "pet_life_stage_at_order": "first"
    }).reset_index()

    simple = grp.merge(rep_cols, on="item_key", how="left")

    # Build final table
    simple["source"] = "tails"
    simple["category"] = simple["pet_food_tier"]
    simple["pet_type"] = np.nan          # unknown here
    simple["pet_size"] = simple["pet_breed_size"]
    simple["price"] = np.nan             # not given directly
    simple["rating_raw"] = np.nan        # no explicit rating

    simple["title_or_name"] = (
        "Tails " + simple["pet_food_tier"].astype(str) +
        " " + simple["wet_tray_size_clean"].astype(str) +
        " (" + simple["pet_life_stage_at_order"].astype(str) +
        ", " + simple["pet_breed_size"].astype(str) + ")"
    )

    simple = simple.rename(columns={
        "score_tails": "global_score"
    })

    # Align with rest of schema
    simple["raw_id"] = np.nan

    # Keep columns consistent + some source-specific metrics
    cols = [
        "item_key", "source", "title_or_name", "category",
        "pet_type", "pet_size", "price", "rating_raw",
        "n_orders", "n_rows", "total_wet_trays", "mean_kcal",
        "mean_discount", "global_score", "raw_id"
    ]
    return simple[cols]


# -------------------------------------------------------------------
# 1) PET STORE RECORDS 2020 (has sales, price, rating, re_buy)
# -------------------------------------------------------------------
def build_store_items():
    path = BASE / "pet-store-records-2020" / "pet_store_records_2020.csv"
    df = pd.read_csv(path)

    df["item_key"] = "store_" + df["product_id"].astype(str)
    df["source"] = "store"

    # Normalize features
    rating_norm = df["rating"] / 10.0  # ratings 1–10
    sales_norm = log1p_norm(df["sales"])
    price_norm = minmax_norm(df["price"])
    rebuy_norm = df["re_buy"].astype(float)  # 0 or 1

    df["score_store"] = (
        0.4 * rating_norm +
        0.35 * sales_norm +
        0.15 * rebuy_norm -
        0.10 * price_norm
    )

    simple = df[[
        "item_key", "source", "product_id", "product_category",
        "pet_type", "pet_size", "price", "rating", "sales", "re_buy",
        "score_store"
    ]].copy()

    simple = simple.rename(columns={
        "product_id": "raw_id",
        "product_category": "category",
        "score_store": "global_score",
        "rating": "rating_raw"
    })

    simple["title_or_name"] = (
        "Store " + simple["category"].astype(str) +
        " (" + simple["pet_type"].astype(str) + ", " +
        simple["pet_size"].astype(str) + ")"
    )

    return simple


# -------------------------------------------------------------------
# 2) ALIEXPRESS (averageStar, quantity, tradeAmount, wishedCount)
# -------------------------------------------------------------------
def build_aliexpress_items():
    path = BASE / "e-commerce-pet-supplies-dataset" / "aliexpress_pet_supplies.csv"
    df = pd.read_csv(path)

    df["item_key"] = "ali_" + df.index.astype(str)
    df["source"] = "aliexpress"

    df["avg_star"] = df["averageStar"].astype(float)
    df["trade_num"] = to_numeric_series(df["tradeAmount"])
    df["wished"] = df["wishedCount"].astype(float)
    df["quantity"] = df["quantity"].astype(float)

    # Normalize
    star_norm = minmax_norm(df["avg_star"])
    trade_norm = log1p_norm(df["trade_num"])
    wish_norm = log1p_norm(df["wished"])
    qty_norm = log1p_norm(df["quantity"])

    df["score_ali"] = (
        0.45 * star_norm +
        0.30 * trade_norm +
        0.15 * wish_norm +
        0.10 * qty_norm
    )

    simple = df[[
        "item_key", "source", "title",
        "avg_star", "trade_num", "wished", "quantity", "score_ali"
    ]].copy()

    simple = simple.rename(columns={
        "title": "title_or_name",
        "score_ali": "global_score"
    })

    # Map avg_star as rating_raw (so we have a common rating column)
    simple["rating_raw"] = simple["avg_star"]

    # Other standard fields
    simple["price"] = np.nan
    simple["category"] = np.nan
    simple["pet_type"] = np.nan
    simple["pet_size"] = np.nan
    simple["raw_id"] = np.nan

    return simple


# -------------------------------------------------------------------
# 3) CHEWY (average_rating, reviews_count, Price)
# -------------------------------------------------------------------
def build_chewy_items():
    path = BASE / "chewy-data" / "chewy_scraper_sample.csv"
    df = pd.read_csv(path)

    df["item_key"] = "chewy_" + df["sku"].astype(str)
    df["source"] = "chewy"

    df["price_val"] = to_numeric_series(df["Price"])
    df["rating_val"] = df["average_rating"].astype(float)
    df["reviews_val"] = df["reviews_count"].fillna(0).astype(float)

    rating_norm = minmax_norm(df["rating_val"])
    reviews_norm = log1p_norm(df["reviews_val"])
    price_norm = minmax_norm(df["price_val"])

    df["score_chewy"] = (
        0.5 * rating_norm +
        0.35 * reviews_norm -
        0.15 * price_norm
    )

    simple = df[[
        "item_key", "source", "name", "brand", "breadcrumb",
        "price_val", "rating_val", "reviews_val", "score_chewy"
    ]].copy()

    simple = simple.rename(columns={
        "name": "title_or_name",
        "price_val": "price",
        "rating_val": "rating_raw",
        "score_chewy": "global_score",
        "breadcrumb": "category"
    })

    simple["pet_type"] = np.nan
    simple["pet_size"] = np.nan
    simple["raw_id"] = df["sku"].astype(str)

    return simple


# -------------------------------------------------------------------
# 4) AMAZON (only price & brand; no rating in sample)
# -------------------------------------------------------------------
def build_amazon_items():
    path = BASE / "amazon-pet-supplies-data" / "amazon_pet_supplies_dataset_sample.csv"
    df = pd.read_csv(path)

    df["item_key"] = "amz_" + df["asin"].astype(str)
    df["source"] = "amazon"

    df["price_val"] = to_numeric_series(df["price"])
    rating = pd.Series(np.zeros(len(df)), index=df.index, dtype=float)

    price_norm = minmax_norm(df["price_val"])
    rating_norm = rating  # all zeros

    df["score_amz"] = (
        0.2 * rating_norm -
        0.8 * price_norm
    )

    simple = df[[
        "item_key", "source", "title", "brand",
        "breadcrumbs", "price_val", "score_amz"
    ]].copy()

    simple = simple.rename(columns={
        "title": "title_or_name",
        "price_val": "price",
        "score_amz": "global_score",
        "breadcrumbs": "category"
    })

    simple["rating_raw"] = rating
    simple["pet_type"] = np.nan
    simple["pet_size"] = np.nan
    simple["raw_id"] = df["asin"].astype(str)

    return simple


# -------------------------------------------------------------------
# 5) Build one unified item table (ALL SOURCES)
# -------------------------------------------------------------------
def build_item_only_catalog():
    tails_items = build_tails_items()
    store_items = build_store_items()
    ali_items = build_aliexpress_items()
    chewy_items = build_chewy_items()
    amz_items = build_amazon_items()

    all_items = pd.concat(
        [tails_items, store_items, ali_items, chewy_items, amz_items],
        ignore_index=True,
        sort=False
    )

    # Re-normalize global_score within each source
    all_items["source_score_norm"] = (
        all_items.groupby("source")["global_score"]
        .transform(lambda x: minmax_norm(x))
    )

    # Final combined score: 50% global_score, 50% source-normalized
    all_items["final_score"] = (
        0.5 * minmax_norm(all_items["global_score"]) +
        0.5 * all_items["source_score_norm"]
    )

    return all_items


# -------------------------------------------------------------------
# 6) Recommendation functions (no users; just item ranking)
# -------------------------------------------------------------------
class ItemOnlyRecommender:
    """
    No user dimension: just rank items by final_score.
    You can filter by source, price, pet_type, etc.
    """
    def __init__(self, items_df: pd.DataFrame):
        self.items = items_df.copy()
        self.items = self.items.replace([np.inf, -np.inf], np.nan)
        self.items["final_score"] = self.items["final_score"].fillna(0.0)

    def recommend_global(
        self,
        N: int = 10,
        source: str | None = None,
        max_price: float | None = None,
        pet_type: str | None = None
    ) -> pd.DataFrame:
        df = self.items

        if source is not None:
            df = df[df["source"] == source]

        if max_price is not None:
            df = df[(df["price"].notna()) & (df["price"] <= max_price)]

        if pet_type is not None:
            pt = pet_type.lower()
            mask = (
                df["pet_type"].fillna("").str.lower().str.contains(pt) |
                df["category"].fillna("").str.lower().str.contains(pt) |
                df["title_or_name"].fillna("").str.lower().str.contains(pt)
            )
            df = df[mask]

        df = df.sort_values("final_score", ascending=False)
        return df.head(N).reset_index(drop=True)


# -------------------------------------------------------------------
# 7) Example usage
# -------------------------------------------------------------------
if __name__ == "__main__":
    items_catalog = build_item_only_catalog()
    print("Unified item-only catalog:", items_catalog.shape)
    print(items_catalog.head(5)[[
        "item_key", "source", "title_or_name",
        "price", "rating_raw", "global_score", "final_score"
    ]])

    recsys = ItemOnlyRecommender(items_catalog)

    print("\n=== TOP 10 GLOBAL ITEMS (all sources) ===")
    print(
        recsys.recommend_global(N=10)[[
            "item_key", "source", "title_or_name",
            "price", "rating_raw", "final_score"
        ]]
    )

    print("\n=== TOP 5 CHEAP STORE ITEMS FOR DOGS (price <= 10000) ===")
    print(
        recsys.recommend_global(
            N=5, source="store", pet_type="dog", max_price=10000
        )[
            ["item_key", "title_or_name", "price",
             "rating_raw", "sales", "re_buy", "final_score"]
        ]
    )

    print("\n=== TOP 5 ALIEXPRESS ITEMS ===")
    print(
        recsys.recommend_global(N=5, source="aliexpress")[
            ["item_key", "source", "title_or_name",
             "rating_raw", "trade_num", "wished", "quantity", "final_score"]
        ]
    )

    print("\n=== TOP 5 TAILS ITEMS ===")
    print(
        recsys.recommend_global(N=5, source="tails")[
            ["item_key", "title_or_name",
             "n_orders", "total_wet_trays", "mean_kcal",
             "mean_discount", "final_score"]
        ]
    )


Unified item-only catalog: (3557, 25)
                              item_key source  \
0   tails_mid|150g|half_maturity|giant  tails   
1   tails_mid|150g|half_maturity|large  tails   
2  tails_mid|150g|half_maturity|medium  tails   
3   tails_mid|150g|half_maturity|small  tails   
4     tails_mid|150g|half_maturity|toy  tails   

                            title_or_name  price  rating_raw  global_score  \
0   Tails mid 150g (half_maturity, giant)    NaN         NaN      0.156525   
1   Tails mid 150g (half_maturity, large)    NaN         NaN      0.365662   
2  Tails mid 150g (half_maturity, medium)    NaN         NaN      0.376695   
3   Tails mid 150g (half_maturity, small)    NaN         NaN      0.470850   
4     Tails mid 150g (half_maturity, toy)    NaN         NaN      0.364670   

   final_score  
0     0.385000  
1     0.573337  
2     0.583273  
3     0.668064  
4     0.572444  

=== TOP 10 GLOBAL ITEMS (all sources) ===
     item_key      source                            

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan